**IMPORTAMOS LIBRERIAS**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import plot_tree
from sklearn import metrics
import numpy as np

**IMPORTAMOS CSV**

In [17]:
train_c = pd.read_csv("../Data/train_c.csv")
test_c = pd.read_csv("../Data/test_c.csv")

In [3]:
train_c.head()

,id,carat,cut,color,clarity,depth,table,price
0,0,1.14,5,4,6,61.0,56.0,9013
1,1,0.76,5,3,4,62.7,57.0,2692
2,2,0.84,5,4,5,61.4,56.0,4372
3,3,1.55,5,3,5,62.0,57.0,13665
4,4,0.30,5,4,2,61.9,57.0,422


In [19]:
# seleccionamos predictor y resultados

X = train_c.drop(['id', 'price'], axis = 1)
y = train_c.price

In [5]:
# creamos train_test_split
X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2)

In [14]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(test_c.shape)

(32364, 6) (8091, 6) (32364,) (8091,)
(13485, 7)


**PROBAREMOS VARIOS MODELOS DE ML**

In [7]:
models = {
    'Linear': LinReg(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'SGD': SGDRegressor(),
    'KN': KNeighborsRegressor(),
    'Grad': GradientBoostingRegressor(),
    'DTree': DecisionTreeRegressor(),
    'RForest': RandomForestRegressor(),}

In [8]:
for name, model in models.items():
    print("Training: ", name)
    model.fit(X_train, y_train)

Training:  Linear
Training:  Ridge
Training:  Lasso
Training:  SGD
Training:  KN
Training:  Grad
Training:  DTree
Training:  RForest


In [9]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f'RMSE {model} =>   ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE LinearRegression() =>    1238.0612596552446
RMSE Ridge() =>    1238.0608982608783
RMSE Lasso() =>    1238.0615060176128
RMSE SGDRegressor() =>    9771820.544022659
RMSE KNeighborsRegressor() =>    1890.7237034148595
RMSE GradientBoostingRegressor() =>    636.4397860932454
RMSE DecisionTreeRegressor() =>    737.6260648327305
RMSE RandomForestRegressor() =>    578.4409681899039


**NUESTRO MEJOR ERROR LO ENCONTRAMOS EN:           
    - RandomForestRegressor() => RMSE :  578.4409681899039**

In [10]:
forest = RandomForestRegressor()
forest.fit(X_train,y_train)

RandomForestRegressor()

In [23]:
test_c.drop(["id"], axis=1, inplace=True)

In [24]:
y_pred = forest.predict(test_c)

In [29]:
test_c["y_pred"] = y_pred

In [28]:
submission1 = test_c[["y_pred"]]
submission1.head()

,y_pred
0,3233.35
1,2998.19
2,3272.99
3,3209.01
4,5296.38


In [30]:
test_c.to_csv("../Data/submission1.csv")